In [1]:
import pandas as pd
import numpy as np
import random

## Kai's Code

In [0]:
glob_pop = []

# states
states = ['well', 'infected-a', 'infected-s', 'recovered', 'dead']
class dude:
      def __init__(self, occupation, age, state=0, underlying_conditions=False, network=[]):
        self.job = occupation
        self.age = age
        self.state = state
        self.under_cond = underlying_conditions
        self.network = {'john': 'near', 'james':'far'}
        self.network = [groupa]
        
class group:
    def __init__(self):
        self.transmission_rate = 0.0
        self.demographics = [0, 0, 0]
        self.infected_count = 0
        self.memebers = []
        
        

Eddie = dude('developer', age=21)
Kai = dude('student', age=21)
Sihan = dude(occupation='student', age=22, state=0)



## Eddie's Code

In [22]:
glob_pop = []

# states
states = ['well', 'infected-a', 'infected-s', 'recovered', 'dead']
jobs = ['student', 'doctor', 'lawyer', 'police person', 'shop assistant', 'unemployed', 'retired']
job_type = ['student', 'high-income', 'low-income', 'retired']
class dude:
    def __init__(self, occupation, age, name, state=0, underlying_conditions=False, network=[]):
        self.job = occupation
        self.name = name
        self.age = age
        self.state = state
        self.under_cond = underlying_conditions
        self.network = network
    
    def __str__(self):
        return "Job: {}, Age: {}, State: {}".format(self.job, self.age, self.state)
        
    def __repr__(self):
        return "Job: {}, Age: {}, State: {}".format(self.job, self.age, self.state)
        
        
class group:
    def __init__(self, size, rate, code, type_):
        self.code = code
        self.type = type_
        self.transmission_rate = rate
        self.demographics = [0, 0, 0]
        self.infected_count = 0
        self.size = size
        self.members = []
        
    def __str__(self):
        return "Size: {}, Infected: {}".format(self.size, self.infected_count)
    
    def __repr__(self):
        return "Code: {}, Size: {}, Infected: {}".format(self.code, self.size, self.infected_count)
        
    def display(self):
        print("Code: {}, Size: {}, Infected: {}".format(self.code, self.size, self.infected_count))
        return

def get_group(g_list, code):
    for grp in g_list:
        if grp.code == code:
            grp.display()
            return grp

Initial population creation and first circle

In [23]:
comm_size = 1000

houses = []
for i in range(0, comm_size):
    rate = random.random()
    size = random.randint(1, 7)
    grp = group(size, rate, i, 'inner')
    houses.append(grp)

In [24]:
peeps = []
for i in range(0, len(houses)):
    job = random.choice(job_type)
    age = random.randint(18, 80)
    person = dude(job, age, i, network=[houses[i]])
    houses[i].members.append(person)
    peeps.append(person)

In [25]:
pep_cnt = len(peeps)
for house in houses:
    job = house.members[0].job
    
    for new_member in range(0, house.size -1):
        age = abs(house.members[0].age + random.randint(-5, 5))
        person = dude(job, age, pep_cnt, network=[house])
        house.members.append(person)
        peeps.append(person)
        pep_cnt +=1
        

Second Circle

In [26]:
# Size of outer circles, dependent on populaiton size
ocircle_size = int(len(peeps) * 0.008 + random.randint(-5, 5))

In [27]:
# Number of outer circles, dependent on inner circle count
ocircle_count = int((len(peeps) / ocircle_size) * 0.95)

In [28]:
ocircle_size * ocircle_count

3782

In [29]:
ocircs = []
for i in range(0, ocircle_count):
    # Lower rates for outr circles
    rate = random.uniform(0,0.5)
    size = int(len(peeps) * 0.008 + random.randint(-5, 5))
    grp = group(size, rate, i, 'outer')
    ocircs.append(grp)

In [30]:
for pepe in peeps:
    nets = random.sample(ocircs, random.randint(1,3))
    for net in nets:
        net.members.append(pepe)
        pepe.network.append(net)

Explore the Networks

In [11]:
pepsam = random.sample(peeps, 5)

for pep in pepsam:
    print(pep.name, pep)
    for net in pep.network:
        print(net.code, net)

3333 Job: student, Age: 46, State: 0
786 Size: 7, Infected: 0
106 Size: 32, Infected: 0
76 Size: 33, Infected: 0
3701 Job: high-income, Age: 69, State: 0
911 Size: 5, Infected: 0
65 Size: 27, Infected: 0
52 Size: 26, Infected: 0
3004 Job: student, Age: 21, State: 0
674 Size: 3, Infected: 0
99 Size: 27, Infected: 0
52 Size: 26, Infected: 0
1273 Job: retired, Age: 38, State: 0
90 Size: 3, Infected: 0
100 Size: 30, Infected: 0
40 Size: 28, Infected: 0
1426 Job: high-income, Age: 59, State: 0
144 Size: 5, Infected: 0
68 Size: 32, Infected: 0
63 Size: 30, Infected: 0
30 Size: 34, Infected: 0


In [31]:
get_group(ocircs, 30)

Code: 30, Size: 29, Infected: 0


Code: 30, Size: 29, Infected: 0

In [14]:
for cir in ocircs:
    if cir.code == 30:
        cir.display()

Code: 30, Size: 34, Infected: 0


In [178]:
print(random.choice(random.choice(ocircs).members))

Job: high-income, Age: 60, State: 0


In [180]:
print(peeps[5].network[0])

Size: 2, Infected: 0
